# 3.3 Black-Derman-Toy Valuation
- last updated : 2023-09-09

Black-Derman-Toy 가치평가를 수행한다. 
단,본실습은 기초자산, 이자율 등 필요 변수들을 설명의 편의상 앞선 엑셀실습과 동일하게 정의하였다.

In [1]:

# input
node_count = 6 # 노드개수
coupon_rate = 0.01 # 액면이자율
issue_price = 10000 # 발행가격
spot_rates = [0.04 , 0.045, 0.05, 0.055, 0.06] # 현물이자율
yield_to_maturity = 1.08  # 만기보장수익률
put_yield = 1.03  # 조기상환수익률
p = 0.5
q = 1 - p
interest_volatility = 0.2 # 이자율변동성

- Step 1. 노드배열 생성
- Step 2. 부트스트래핑을 사용하여 kd 및 rf spot rate 및 forward rate 계산
- Step 3: 상환전환우선주 상환금액 트리 생성
- Step 4: Rd를 구하기 위한 calibration 설정
- Step 5: 조기상환청구트리, 사채가치트리 생성
- Step 6: BDT밸류에이션


### Step 1. 노드배열 생성


In [15]:
import numpy as np

# 노드배열은 0시점 ~ 5시점까지 6개 노드로 가정한다.
node_array = np.arange(0, node_count)
print(node_array)

# 출력: 
"""
[0 1 2 3 4 5]
"""

[0 1 2 3 4 5]


'\n[0 1 2 3 4 5]\n'

### Step 2. 부트스트래핑을 사용하여 위험이자율과 무위험이자율의 현물이자율 및 선도이자율 계산 

In [16]:
forward_rates = [spot_rates[0]]
for i in range(1, len(spot_rates)):
    t1 = i + 1
    t2 = i 
    
    forward_rate = ((np.power((1 + spot_rates[i]), t1) / np.power((1 + spot_rates[i - 1] ), t2)) - 1)
    forward_rates.append(forward_rate)
    

# 다른 배열과의 크기를 맞추기 위한 과정이며, 현재시점의 이자율을 0.0으로 초기화한다. 
spot_rates.insert(0, 0)
forward_rates.insert(0, 0)

print(forward_rates)

# 출력
"""
[0, 0.04, 0.050024038461538245, 0.06007188480117254, 0.0701433111974945, 0.08023809255397985]
"""

[0, 0.04, 0.050024038461538245, 0.06007188480117254, 0.0701433111974945, 0.08023809255397985]


'\n[0, 0.04, 0.050024038461538245, 0.06007188480117254, 0.0701433111974945, 0.08023809255397985]\n'

### Step 3: 액면이자, 만기할증금, 조기상환청구금액 배열 산출

In [17]:
# 액면이자율 배열 산출, 발행가 x 액면이자율
coupon_array = np.full(node_count, coupon_rate * issue_price)
coupon_array[0] = 0 

# 보장수익률을 반영한 만기시점의 만기할증금 배열 산출 
maturity_ammount_array = np.zeros_like(node_array)
maturity_ammount_array[-1] = issue_price * yield_to_maturity

# 조기상환청구금액배열 생성, 4시점에 조기상환이 가능하다고 가정
put_ammount_array = np.zeros_like(node_array)
put_ammount_array[4] = issue_price * put_yield + coupon_array[4]

### Step 4: Rd를 구하기 위한 calibration 설정

In [18]:
from scipy.optimize import fsolve

interest_rate_tree = np.zeros((node_count, node_count))

# 첫번째 시점의 이자율은 첫번째 시점의 선도이자율과 같다.
interest_rate_tree[0, 1] = forward_rates[1]


# 두번째 시점에서 마지막시점까지 for-loop
for i in range(2, node_count):

    # 각 시점별 조정(calibration)트리 크기를 초기화 한다. 
    calibration_tree = np.zeros((i-1, i-1))    
    rd_step = np.exp(2 * interest_volatility)
    square_array = np.arange(i-1, -1, -1)
    
    def equation(rd):
    
        interest_rate_tree[:i, i] = (rd_step ** square_array) * rd

        # i보다 1개 낮은 배열로 초기화

        for j in reversed(range(1, i)):
            
            # 가장 마지막 노드인 경우
            if i - 1 == j:
                calibration_tree[:j, j-1] = (1 / (1 + interest_rate_tree[:j, j+1]) + 1 / (1 + interest_rate_tree[1:j+1, j+1]))*0.5
            
            # 가장 마지막 노드 이외의 경우
            else:
                calibration_tree[:j, j-1] = (calibration_tree[:j, j] / (1 + interest_rate_tree[:j, j+1]) + calibration_tree[1:j+1, j] / (1 + interest_rate_tree[1:j+1, j+1])) * 0.5

        return (calibration_tree[0, 0] / (1 + interest_rate_tree[0, 1])) - (1 / np.power((1 + spot_rates[i]), node_array[i]))

    # 해찾기의 첫번째 dummy값을 해당 기간의 선도이자율로 초기화한다.
    rd_dummy = forward_rates[i]

    # fsolve 메소드로 equation 함수를 통해서 해찾기
    rd_adjusted = fsolve(equation, rd_dummy)
    
    # 실제로 구해진 rd값을 이용하여, exp(volatility * 2) 만큼 상승시킨 노드값들을 구한다.
    interest_rate_tree[:i, i] = (rd_step ** square_array) * rd_adjusted


In [19]:
interest_rate_tree

array([[0.        , 0.04      , 0.06000912, 0.08667598, 0.12215156,
        0.16936208],
       [0.        , 0.        , 0.04022532, 0.05810065, 0.08188064,
        0.1135268 ],
       [0.        , 0.        , 0.        , 0.03894603, 0.05488623,
        0.07609929],
       [0.        , 0.        , 0.        , 0.        , 0.03679134,
        0.05101088],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03419361],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

### Step5. 사채가치트리 생성

In [20]:
# 사채가치트리를 생성한다. 
discounted_bond_tree = np.zeros_like(interest_rate_tree)

# 사채가치트리의 마지막 노드는 마지막시점의 보장수익률이 반영된 원리금
discounted_bond_tree[:, -1] = maturity_ammount_array[-1] + coupon_array[-1]

# 마지막부터 백워데이션 방식을 통해서, 각 노드별로 이자율트리에 해당하는 이자율로 할인하여 사채가치트리를 생성한다.
for t in reversed(range(discounted_bond_tree.shape[1] - 1)):
    discounted_bond_tree[:t+1, t] = (discounted_bond_tree[:t+1, t+1] + discounted_bond_tree[1:t+2, t+1])*0.5 / (1 + interest_rate_tree[:t+1, t+1]) + coupon_array[t]


In [21]:
discounted_bond_tree

array([[ 8499.9463597 ,  8426.99649201,  8428.92013605,  8704.02486027,
         9421.32160555, 10900.        ],
       [    0.        ,  9252.89193617,  9224.46431608,  9397.6500147 ,
         9888.71818944, 10900.        ],
       [    0.        ,     0.        ,  9817.67550206,  9911.55315617,
        10229.17686327, 10900.        ],
       [    0.        ,     0.        ,     0.        , 10280.72756395,
        10470.9678301 , 10900.        ],
       [    0.        ,     0.        ,     0.        ,     0.        ,
        10639.6125588 , 10900.        ],
       [    0.        ,     0.        ,     0.        ,     0.        ,
            0.        , 10900.        ]])

### Step 6: BDT밸류에이션

In [26]:
bdt_valuation_tree = np.zeros_like(discounted_bond_tree)
bdt_valuation_tree[:, -1] = discounted_bond_tree[:, -1]

for t in reversed(range(bdt_valuation_tree.shape[1] - 1)):
    holding_value = (bdt_valuation_tree[:t+1, t+1] + bdt_valuation_tree[1:t+2, t+1])*0.5 / (1 + interest_rate_tree[:t+1, t+1]) + coupon_array[t]
    discounted_bond_value = discounted_bond_tree[:t+1, t]

    bdt_valuation_tree[:t+1, t] = np.maximum(holding_value, np.maximum(put_ammount_array[t], discounted_bond_value))

print(bdt_valuation_tree[:3,:3])

# 출력:
"""
[[8696.8410545  8727.81474907 8879.43430433]
 [   0.         9361.61464428 9411.69033986]
 [   0.            0.         9856.64170515]]
"""

[[8696.8410545  8727.81474907 8879.43430433]
 [   0.         9361.61464428 9411.69033986]
 [   0.            0.         9856.64170515]]


'\n[[ 8696.8410545   8727.81474907  8879.43430433  9367.910319910400.         10900.        ]\n [    0.          9361.61464428  9411.69033986  9712.89040424\n  10400.         10900.        ]\n [    0.             0.          9856.64170515  9992.52071998\n  10400.         10900.        ]\n [    0.             0.             0.         10280.72756395\n  10470.9678301  10900.        ]\n [    0.             0.             0.             0.\n  10639.6125588  10900.        ]\n [    0.             0.             0.             0.\n      0.         10900.        ]]\n'

In [192]:
bdt_valuation_tree

array([[ 8696.8410545 ,  8727.81474907,  8879.43430433,  9367.9103199 ,
        10400.        , 10900.        ],
       [    0.        ,  9361.61464428,  9411.69033986,  9712.89040424,
        10400.        , 10900.        ],
       [    0.        ,     0.        ,  9856.64170515,  9992.52071998,
        10400.        , 10900.        ],
       [    0.        ,     0.        ,     0.        , 10280.72756395,
        10470.9678301 , 10900.        ],
       [    0.        ,     0.        ,     0.        ,     0.        ,
        10639.6125588 , 10900.        ],
       [    0.        ,     0.        ,     0.        ,     0.        ,
            0.        , 10900.        ]])